In [3]:
###fit the regression 
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
import math
from matplotlib import pyplot as plt
%matplotlib inline

filepath = "/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/code/"

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
from sklearn import linear_model
clf = linear_model.LinearRegression()

print("LINEAR REGRESSION!")

train = pd.read_csv(filepath + 'jan_may_2017_links_cross_posting_info.csv')
##drop dups
# train = train.sort_values('ext_table1_firsthour_stats_date', ascending = False).drop_duplicates(subset=['buzz_post_buzz_id', 'ext_external_id'], keep = 'last')
all_pages = list(train.buzz_account_display_name.unique())

####RESTRICT PAGE-WISE
# train = train[train['buzz_account_display_name'] == 'BuzzFeed']

# print("WITHOUT cross-promotion features")

features = [
       'ext_table1_consumptions_by_type__link_clicks',
       'ext_table1_consumptions_by_type__other_clicks',
       'ext_table1_reactions_like_total',
       'ext_table1_stories_by_action_type__comment',
       'ext_table1_stories_by_action_type__like',
       'ext_table1_stories_by_action_type__share', 'ext_table1_engaged_fan',
       'ext_table1_fan_reach', 'ext_table1_impressions',
       'ext_table1_impressions_fan',
        ]

    
alt_inf = len(all_pages) + 1
train = train.replace(math.inf, alt_inf)

for feat in features:
    train[feat] = np.log(train[feat] + 1)

# ####add the category variable for each of the pages in col_pages
# for pg in all_pages:
#     features.append(pg)

###generate DFs for analysis - X and Y
df = train[features]

train['clicks_bucket'] = np.log(train['ext_twodays_consumptions_by_type__link_clicks'] + 1)
train['share_bucket'] = np.log(train['ext_twodays_stories_by_action_type__share'] + 1)

y_clicks = train['clicks_bucket']
y_share = train['share_bucket']

# y_clicks = train['ext_twodays_consumptions_by_type__link_clicks']
# y_share = train['ext_twodays_stories_by_action_type__share']

X = train[features]

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

for train_index, test_index in kf.split(X):
    print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
        
    accuracy = metrics.r2_score(y_test, y_pred)
    print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
    print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
    print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    accuracy = metrics.r2_score(y_test, y_pred)
    print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
    print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

LINEAR REGRESSION!
Shares: Fold =  0
Cross-Predicted Accuracy (R2): 0.894169412975
Mean values (share):  3.6573258659506345  | percent error:  0.130517293182
Clicks: Fold =  0
Cross-Predicted Accuracy (R2): 0.901915549722
Mean values (share):  8.335194671014994  | percent error:  0.0499733184156
Shares: Fold =  1
Cross-Predicted Accuracy (R2): 0.901192295147
Mean values (share):  3.663375574408008  | percent error:  0.128871611172
Clicks: Fold =  1
Cross-Predicted Accuracy (R2): 0.918218495433
Mean values (share):  8.362863312461617  | percent error:  0.0472612687475
Shares: Fold =  2
Cross-Predicted Accuracy (R2): 0.900469142309
Mean values (share):  3.6574683179896534  | percent error:  0.129279711901
Clicks: Fold =  2
Cross-Predicted Accuracy (R2): 0.911176896421
Mean values (share):  8.36676476126156  | percent error:  0.0478832958281
Shares: Fold =  3
Cross-Predicted Accuracy (R2): 0.901217480653
Mean values (share):  3.712750800179595  | percent error:  0.127532386701
Clicks: Fol

In [6]:
from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(max_depth=10)

print("TREE REGRESSION!")

train = pd.read_csv(filepath+'jan_may_2017_links_cross_posting_info.csv')
##drop dups
# train = train.sort_values('ext_table1_firsthour_stats_date', ascending = False).drop_duplicates(subset=['buzz_post_buzz_id', 'ext_external_id'], keep = 'last')
all_pages = list(train.buzz_account_display_name.unique())

####RESTRICT PAGE-WISE
# train = train[train['buzz_account_display_name'] == 'BuzzFeed']

# print("WITHOUT cross-promotion features")

features = [
       'ext_table1_consumptions_by_type__link_clicks',
       'ext_table1_consumptions_by_type__other_clicks',
       'ext_table1_reactions_like_total',
       'ext_table1_stories_by_action_type__comment',
       'ext_table1_stories_by_action_type__like',
       'ext_table1_stories_by_action_type__share', 'ext_table1_engaged_fan',
       'ext_table1_fan_reach', 'ext_table1_impressions',
       'ext_table1_impressions_fan',
        ]

    
alt_inf = len(all_pages) + 1
train = train.replace(math.inf, alt_inf)

# ####add the category variable for each of the pages in col_pages
# for pg in all_pages:
#     features.append(pg)

###generate DFs for analysis - X and Y
df = train[features]

train['clicks_bucket'] = train['ext_twodays_consumptions_by_type__link_clicks']
train['share_bucket'] = train['ext_twodays_stories_by_action_type__share']

# y_clicks = train['clicks_bucket']
# y_share = train['share_bucket']

y_clicks = train['ext_twodays_consumptions_by_type__link_clicks']
y_share = train['ext_twodays_stories_by_action_type__share']

X = train[features]
y = y_clicks

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

for train_index, test_index in kf.split(X):
    print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
        
    accuracy = metrics.r2_score(y_test, y_pred)
    print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
    print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
    print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    accuracy = metrics.r2_score(y_test, y_pred)
    print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
    print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

TREE REGRESSION!
Shares: Fold =  0
Cross-Predicted Accuracy (R2): 0.959296819598
Mean values (share):  271.57089151450054  | percent error:  0.231732180023
Clicks: Fold =  0
Cross-Predicted Accuracy (R2): 0.947180100743
Mean values (share):  26358.91111707841  | percent error:  0.208781183334
Shares: Fold =  1
Cross-Predicted Accuracy (R2): 0.964132719057
Mean values (share):  317.96776792908946  | percent error:  0.251024909004
Clicks: Fold =  1
Cross-Predicted Accuracy (R2): 0.960440213313
Mean values (share):  28609.518130539887  | percent error:  0.215223561438
Shares: Fold =  2
Cross-Predicted Accuracy (R2): 0.936007868333
Mean values (share):  293.42680633897396  | percent error:  0.26474723521
Clicks: Fold =  2
Cross-Predicted Accuracy (R2): 0.926302530134
Mean values (share):  25356.115498254097  | percent error:  0.215551240738
Shares: Fold =  3
Cross-Predicted Accuracy (R2): 0.943699730263
Mean values (share):  290.64920762825676  | percent error:  0.259397838662
Clicks: Fold

In [9]:
print('Share performance')
res = ols(y = y_share, x = train[features])
print(res)

print('Clicks performance')
res = ols(y = y_clicks, x = train[features])
print(res)

Share performance


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <ext_table1_consumptions_by_type__link_clicks>
             + <ext_table1_consumptions_by_type__other_clicks> + <ext_table1_reactions_like_total>
             + <ext_table1_stories_by_action_type__comment>
             + <ext_table1_stories_by_action_type__like> + <ext_table1_stories_by_action_type__share>
             + <ext_table1_engaged_fan> + <ext_table1_fan_reach>
             + <ext_table1_impressions> + <ext_table1_impressions_fan> + <intercept>

Number of Observations:         37231
Number of Degrees of Freedom:   11

R-squared:         0.7657
Adj R-squared:     0.7656

Rmse:            499.1729

F-stat (10, 37220): 12161.3910, p-value:     0.0000

Degrees of Freedom: model 10, resid 37220

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
---------------------------------

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)
